# Adversarial example using pytorch

In this example, we redo the [adversarial example](https://gurobi-optimization-ml2gurobi.readthedocs-hosted.com/en/latest/examples/adversarial_mnist.html) of the documentation but use tensorflow Keras for training the neural network.

We don't detail the optimization model here. Please refer to the example in the documentation.
Note that many of the differences between this notebook and the one from the documentation come from
using tensorflow instead of numpy for manipulating data.

This example requires the additional packages:
 - [tensorflow](https://www.tensorflow.org/)
 - [matplotlib](https://matplotlib.org/)

## Import the necessary packages and load data

We import all the package we need for this example.
The MNIST dataset is available from Keras.

In [ ]:
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

import gurobipy as gp

from gurobi_ml import add_predictor_constr

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

We reshape and scale `x_train` and `x_test`.

In [ ]:
x_train = tf.reshape(tf.cast(x_train, tf.float32) / 255.0, [-1, 28 * 28])
x_test = tf.reshape(tf.cast(x_test, tf.float32) / 255.0, [-1, 28 * 28])

## Construct and train the neural network

We construct a sequential neural network with 2 hidden layers of 50 neurons and ReLU activation.

We use the usual Keras functions to compile and fit the network.

In [ ]:
nn = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer(28 * 28),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(10),
    ]
)

In [ ]:
nn.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
nn.fit(
    x_train,
    y_train,
    epochs=6,
    validation_data=(x_test, y_test),
)

Print summary of the trained network

In [ ]:
nn.summary()

## Build optimization model

Now we turn to building the optimization model.

We choose a training example and the steps are similar to the one with scikit-learn in the documentation.
The only differences come from the data being stored in tensors instead of arrays.

In [ ]:
example = x_train[18, :]
plt.imshow(tf.reshape(example, [28, 28]), cmap="gray")
plt.show()
label = tf.math.argmax(nn.predict(tf.reshape(example, (1, -1))), axis=1)
print(f"Example is classified as {label}")

In [ ]:
ex_prob = nn.predict(tf.reshape(example, (1, -1)))
sorted_labels = tf.argsort(ex_prob)[0]
right_label = sorted_labels[-1]
wrong_label = sorted_labels[-2]

In [ ]:
m = gp.Model()
delta = 5

x = m.addMVar(example.numpy().shape, lb=0.0, ub=1.0, name="x")
y = m.addMVar(ex_prob.shape, lb=-gp.GRB.INFINITY, name="y")

abs_diff = m.addMVar(example.numpy().shape, lb=0, ub=1, name="abs_diff")

m.setObjective(y[0, wrong_label] - y[0, right_label], gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-1
m.addConstr(abs_diff >= x - example.numpy())
m.addConstr(abs_diff >= -x + example.numpy())
m.addConstr(abs_diff.sum() <= delta)

# Update the model
m.update()

pred_constr = add_predictor_constr(m, nn, x, y)

pred_constr.print_stats()

In [ ]:
m.Params.BestBdStop = 0.0
m.Params.BestObjStop = 0.0
m.optimize()

Finally, display the adversarial example if one was found.

In [ ]:
if m.ObjVal > 0.0:
    pixels = x.X.reshape((28, 28))
    plt.imshow(pixels, cmap="gray")
    plt.show()
    label = tf.math.argmax(nn.predict(tf.reshape(x.X, (1, -1))), axis=1)
    print(f"Solution is classified as {label}")
else:
    print("No counter example exists in neighborhood.")

copyright © 2022 Gurobi Optimization, LLC